In [1]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


numpy.random.seed(7)

Using TensorFlow backend.
c:\program files\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files\python37\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [3]:
#df = read_csv('flights.csv', usecols=[1], engine='python', skipfooter=3)
df= read_csv('./서울시 기간별 시간평균 대기환경 정보(초미세먼지).csv', encoding='euc-kr')
df.head()

dataset = df['강남구'].values.reshape(-1, 1)
dataset = dataset.astype('float32')

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [5]:
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [6]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [7]:
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

W0818 23:29:08.514714 15096 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 23:29:08.527703 15096 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 23:29:08.528700 15096 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0818 23:29:08.645362 15096 deprecation_wrapper.py:119] From c:\program files\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0818 23:29:08.770034 15096 deprecation.py:323] From c:\program files\python

Epoch 1/100
 - 8s - loss: 0.0124
Epoch 2/100
 - 7s - loss: 0.0044
Epoch 3/100
 - 7s - loss: 0.0042
Epoch 4/100
 - 7s - loss: 0.0040
Epoch 5/100
 - 7s - loss: 0.0039
Epoch 6/100
 - 7s - loss: 0.0038
Epoch 7/100
 - 7s - loss: 0.0037
Epoch 8/100
 - 7s - loss: 0.0035
Epoch 9/100
 - 7s - loss: 0.0034
Epoch 10/100
 - 7s - loss: 0.0033
Epoch 11/100
 - 7s - loss: 0.0031
Epoch 12/100
 - 7s - loss: 0.0029
Epoch 13/100
 - 7s - loss: 0.0028
Epoch 14/100
 - 7s - loss: 0.0027
Epoch 15/100
 - 7s - loss: 0.0027
Epoch 16/100
 - 7s - loss: 0.0027
Epoch 17/100
 - 7s - loss: 0.0027
Epoch 18/100
 - 7s - loss: 0.0026
Epoch 19/100
 - 7s - loss: 0.0026
Epoch 20/100
 - 7s - loss: 0.0026
Epoch 21/100
 - 7s - loss: 0.0026
Epoch 22/100
 - 7s - loss: 0.0026
Epoch 23/100
 - 7s - loss: 0.0025
Epoch 24/100
 - 7s - loss: 0.0025
Epoch 25/100
 - 7s - loss: 0.0025
Epoch 26/100
 - 7s - loss: 0.0026
Epoch 27/100
 - 7s - loss: 0.0025
Epoch 28/100
 - 7s - loss: 0.0025
Epoch 29/100
 - 7s - loss: 0.0025
Epoch 30/100
 - 7s - lo

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
plt.plot(scaler.inverse_transform(dataset), label='Actual Value')
plt.plot(trainPredictPlot, label='Train Data Predice')
plt.plot(testPredictPlot, label='Test Data Predice')
plt.legend()
plt.show()